In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
sns.set() # Definir o tema do Matplotlib como sendo o tema padrão do Seaborn
# set the graphs to show in the jupyter notebook
%matplotlib inline

In C:\Anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.jpeg_quality rcparam was deprecated in Matplo

In [2]:
df = pd.read_csv('winequality-red.csv', sep=";")
#df = pd.read_csv('winequality-white.csv', sep=";")
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [3]:
def calc_min_and_max_range(dados_coluna):
    Q1 = dados_coluna.quantile(q=0.25) # Primeiro quartil
    Q3 = dados_coluna.quantile(q=0.75) # Terceiro quartil

    IQR = Q3 - Q1 # Intervalo interquartílico

    inf = Q1 - 1.5 * IQR
    sup = Q3 + 1.5 * IQR

    return inf, sup

In [4]:
colunas = list(df.drop(['quality'], axis=1).columns)
colunas

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']

In [5]:
df_out=df.copy()

for coluna in colunas:
    inf, sup = calc_min_and_max_range(df_out[coluna])
    df_out = df_out[(df_out[coluna] > inf) & (df_out[coluna] < sup)]

In [6]:
df_out
df_out.shape

(1135, 12)

# Testando com força bruta para achar a melhor combinação de variávéis para regressão linear simples considerando os dados filtrados

In [7]:
# Função que retorna todos os possiveis sub-conjuntos de uma lista ("Power Set")

def powerset(fullset):
  listsub = list(fullset)
  subsets = []
  for i in range(2**len(listsub)):
    subset = []
    for k in range(len(listsub)):            
      if i & 1<<k:
        subset.append(listsub[k])
    subsets.append(subset)        
  return subsets


In [8]:
# Determinando todas combinações de variáveis

pwr_set = powerset(set(colunas))
print(pwr_set)

[[], ['density'], ['volatile acidity'], ['density', 'volatile acidity'], ['pH'], ['density', 'pH'], ['volatile acidity', 'pH'], ['density', 'volatile acidity', 'pH'], ['alcohol'], ['density', 'alcohol'], ['volatile acidity', 'alcohol'], ['density', 'volatile acidity', 'alcohol'], ['pH', 'alcohol'], ['density', 'pH', 'alcohol'], ['volatile acidity', 'pH', 'alcohol'], ['density', 'volatile acidity', 'pH', 'alcohol'], ['residual sugar'], ['density', 'residual sugar'], ['volatile acidity', 'residual sugar'], ['density', 'volatile acidity', 'residual sugar'], ['pH', 'residual sugar'], ['density', 'pH', 'residual sugar'], ['volatile acidity', 'pH', 'residual sugar'], ['density', 'volatile acidity', 'pH', 'residual sugar'], ['alcohol', 'residual sugar'], ['density', 'alcohol', 'residual sugar'], ['volatile acidity', 'alcohol', 'residual sugar'], ['density', 'volatile acidity', 'alcohol', 'residual sugar'], ['pH', 'alcohol', 'residual sugar'], ['density', 'pH', 'alcohol', 'residual sugar'], ['

In [9]:
print(len(pwr_set))

2048


In [10]:
len(colunas)
pwr_set.remove([]) # removendo o conjunto vazio

In [11]:
pwr_set.sort(key=lambda x: len(x))
print(pwr_set)

[['density'], ['volatile acidity'], ['pH'], ['alcohol'], ['residual sugar'], ['total sulfur dioxide'], ['chlorides'], ['fixed acidity'], ['citric acid'], ['free sulfur dioxide'], ['sulphates'], ['density', 'volatile acidity'], ['density', 'pH'], ['volatile acidity', 'pH'], ['density', 'alcohol'], ['volatile acidity', 'alcohol'], ['pH', 'alcohol'], ['density', 'residual sugar'], ['volatile acidity', 'residual sugar'], ['pH', 'residual sugar'], ['alcohol', 'residual sugar'], ['density', 'total sulfur dioxide'], ['volatile acidity', 'total sulfur dioxide'], ['pH', 'total sulfur dioxide'], ['alcohol', 'total sulfur dioxide'], ['residual sugar', 'total sulfur dioxide'], ['density', 'chlorides'], ['volatile acidity', 'chlorides'], ['pH', 'chlorides'], ['alcohol', 'chlorides'], ['residual sugar', 'chlorides'], ['total sulfur dioxide', 'chlorides'], ['density', 'fixed acidity'], ['volatile acidity', 'fixed acidity'], ['pH', 'fixed acidity'], ['alcohol', 'fixed acidity'], ['residual sugar', 'fi

In [12]:
len(pwr_set)

2047

In [13]:
# criando 1 modelo para cada conjunto de variáveis de entrada e guardando tudo num dicionario

Modelos={}
y_out = df_out['quality'] # Variável resposta


for modelo in range(len(pwr_set)):    
    X_out = df_out[pwr_set[modelo]]
    lr_out = LinearRegression()
    lr_out.fit(X_out, y_out)
    Modelos[str(modelo)]=[X_out,lr_out]    

In [14]:
Modelos

{'0': [      density
  0     0.99780
  1     0.99680
  2     0.99700
  3     0.99800
  4     0.99780
  ...       ...
  1594  0.99490
  1595  0.99512
  1596  0.99574
  1597  0.99547
  1598  0.99549
  
  [1135 rows x 1 columns],
  LinearRegression()],
 '1': [      volatile acidity
  0                0.700
  1                0.880
  2                0.760
  3                0.280
  4                0.700
  ...                ...
  1594             0.600
  1595             0.550
  1596             0.510
  1597             0.645
  1598             0.310
  
  [1135 rows x 1 columns],
  LinearRegression()],
 '2': [        pH
  0     3.51
  1     3.20
  2     3.26
  3     3.16
  4     3.51
  ...    ...
  1594  3.45
  1595  3.52
  1596  3.42
  1597  3.57
  1598  3.39
  
  [1135 rows x 1 columns],
  LinearRegression()],
 '3': [      alcohol
  0         9.4
  1         9.8
  2         9.8
  3         9.8
  4         9.4
  ...       ...
  1594     10.5
  1595     11.2
  1596     11.0
  1597     10

In [15]:
# Calculando o score e o erro quadratico de todos modelos e salvando num lista
Score=[]
Square_Error=[]

for i in range(len(Modelos)):
    Score.append(Modelos[str(i)][1].score(Modelos[str(i)][0],y_out))
    Square_Error.append(mean_squared_error(Modelos[str(i)][1].predict(Modelos[str(i)][0]), y_out))
    

In [16]:
# ordenando os modelos de acordo com maior score

sort_index_Score = np.argsort(Score)
sort_index_Score=list(sort_index_Score)
sort_index_Score.reverse()
print(sort_index_Score)

[2046, 2045, 2038, 2013, 2041, 2028, 2009, 1938, 2039, 2015, 2036, 1998, 2019, 1953, 2007, 1994, 1889, 1929, 1991, 1874, 1925, 1753, 1870, 1647, 1992, 1876, 1880, 1662, 1868, 1638, 1634, 1297, 2037, 1999, 2006, 2002, 1917, 1898, 1902, 1703, 2000, 1896, 1904, 1690, 1908, 1718, 1990, 1860, 1863, 1867, 1607, 1626, 2040, 2020, 2024, 1963, 1694, 1367, 1993, 1881, 2008, 1930, 1934, 1763, 1861, 1605, 1611, 1247, 1885, 1672, 1613, 1234, 1869, 1617, 1262, 2014, 1945, 1639, 1643, 1307, 1949, 2043, 1783, 1875, 1654, 2017, 1924, 1749, 1745, 1238, 1432, 1658, 1996, 821, 1327, 1878, 2026, 1951, 1633, 1293, 1289, 886, 1887, 1660, 2004, 1906, 1915, 2001, 1909, 1897, 1695, 1865, 1716, 1913, 1728, 2022, 1615, 1862, 1618, 1606, 1239, 1699, 1961, 1622, 1377, 1272, 1947, 1624, 1243, 1883, 2033, 1974, 1781, 1260, 831, 1670, 1903, 1710, 1689, 1656, 1359, 1894, 1683, 1325, 1958, 1794, 1612, 1254, 1233, 813, 1714, 1667, 1338, 1397, 1363, 1258, 817, 851, 942, 444, 1922, 1739, 1968, 1911, 1723, 1804, 1631, 1726,

In [17]:
# ordenando os modelos de acordo com menor erro quadrático
sort_index_Square_Error = np.argsort(Square_Error)
sort_index_Square_Error=list(sort_index_Square_Error)
print(sort_index_Square_Error)

[2046, 2045, 2038, 2013, 2041, 2028, 2009, 1938, 2039, 2015, 2036, 1998, 2019, 1953, 2007, 1994, 1889, 1929, 1991, 1874, 1925, 1753, 1870, 1647, 1992, 1876, 1880, 1662, 1868, 1638, 1634, 1297, 2037, 1999, 2006, 2002, 1917, 1898, 1902, 1703, 2000, 1896, 1904, 1690, 1908, 1718, 1990, 1860, 1863, 1867, 1607, 1626, 2040, 2020, 2024, 1963, 1694, 1367, 1993, 1881, 2008, 1930, 1934, 1763, 1861, 1605, 1611, 1247, 1885, 1672, 1613, 1234, 1869, 1617, 1262, 2014, 1945, 1639, 1643, 1307, 1949, 2043, 1783, 1875, 1654, 2017, 1924, 1749, 1745, 1238, 1432, 1658, 1996, 821, 1327, 1878, 2026, 1951, 1633, 1293, 1289, 886, 1887, 1660, 2004, 1906, 1915, 2001, 1909, 1897, 1695, 1865, 1716, 1913, 1728, 2022, 1615, 1862, 1618, 1606, 1239, 1699, 1961, 1622, 1377, 1272, 1947, 1624, 1243, 1883, 2033, 1974, 1781, 1260, 831, 1670, 1903, 1710, 1689, 1656, 1359, 1894, 1683, 1325, 1958, 1794, 1612, 1254, 1233, 813, 1714, 1667, 1338, 1397, 1363, 1258, 817, 851, 942, 444, 1922, 1739, 1968, 1911, 1723, 1804, 1631, 1726,

# O Melhor modelo, considerando o maior score e menor erro quadrático, é o modelo completo utilizando dados de todas as variáveis

In [18]:
Modelos['2046'][0]

,density,volatile acidity,pH,alcohol,residual sugar,total sulfur dioxide,chlorides,fixed acidity,citric acid,free sulfur dioxide,sulphates
0,0.99780,0.700,3.51,9.4,1.9,34.0,0.076,7.4,0.00,11.0,0.56
1,0.99680,0.880,3.20,9.8,2.6,67.0,0.098,7.8,0.00,25.0,0.68
2,0.99700,0.760,3.26,9.8,2.3,54.0,0.092,7.8,0.04,15.0,0.65
3,0.99800,0.280,3.16,9.8,1.9,60.0,0.075,11.2,0.56,17.0,0.58
4,0.99780,0.700,3.51,9.4,1.9,34.0,0.076,7.4,0.00,11.0,0.56
...,...,...,...,...,...,...,...,...,...,...,...
1594,0.99490,0.600,3.45,10.5,2.0,44.0,0.090,6.2,0.08,32.0,0.58
1595,0.99512,0.550,3.52,11.2,2.2,51.0,0.062,5.9,0.10,39.0,0.76
1596,0.99574,0.510,3.42,11.0,2.3,40.0,0.076,6.3,0.13,29.0,0.75
1597,0.99547,0.645,3.57,10.2,2.0,44.0,0.075,5.9,0.12,32.0,0.71


In [19]:
Score[2046]

0.3861655304020821

In [20]:
Square_Error[2046]

0.3528579125800755

# Encontrando o melhor modelo de 3 variáveis, considerando o maior score e menor erro quadrático.

In [21]:
modelos_3_var=[]
modelos_2_var=[]

for i,value in Modelos.items():
    if value[0].shape[1]==3:
        modelos_3_var.append(i)
    if value[0].shape[1]==2:
        modelos_2_var.append(i)

In [22]:
for i in sort_index_Square_Error:
    if str(i) in modelos_3_var:
        melhor_modelo_3_var=i
        break
melhor_modelo_3_var

190

In [25]:
Modelos[str(melhor_modelo_3_var)][0]

,volatile acidity,alcohol,sulphates
0,0.700,9.4,0.56
1,0.880,9.8,0.68
2,0.760,9.8,0.65
3,0.280,9.8,0.58
4,0.700,9.4,0.56
...,...,...,...
1594,0.600,10.5,0.58
1595,0.550,11.2,0.76
1596,0.510,11.0,0.75
1597,0.645,10.2,0.71


**O modelo com 3 variáveis com melhor desempenho é exatamente o que tinha sido selecionado considerando as variáveis com maior correlação com variável alvo (alcohol,	volatile acidity,	sulphates)**